In [1]:
import pandas as pd
import numpy as np
import wrds
from pandas.tseries.offsets import *
import pickle as pkl
import pyarrow.feather as feather
from functions import *

In [23]:
###################
# Connect to WRDS #
###################
conn = wrds.Connection(wrds_username='hangsuck')

### TTM functions 

def ttm4(series, df):
    """

    :param series: variables' name
    :param df: dataframe
    :return: ttm4
    """
    lag = pd.DataFrame()
    for i in range(1, 4):
        lag['%(series)s%(lag)s' % {'series': series, 'lag': i}] = df.groupby('permno')['%s' % series].shift(i)
    result = df['%s' % series] + lag['%s1' % series] + lag['%s2' % series] + lag['%s3' % series]
    return result


def ttm12(series, df):
    """

    :param series: variables' name
    :param df: dataframe
    :return: ttm12
    """
    lag = pd.DataFrame()
    for i in range(1, 12):
        lag['%(series)s%(lag)s' % {'series': series, 'lag': i}] = df.groupby('permno')['%s' % series].shift(i)
    result = df['%s' % series] + lag['%s1' % series] + lag['%s2' % series] + lag['%s3' % series] +             lag['%s4' % series] + lag['%s5' % series] + lag['%s6' % series] + lag['%s7' % series] +             lag['%s8' % series] + lag['%s9' % series] + lag['%s10' % series] + lag['%s11' % series]
    return result


### Compustat Block
comp = conn.raw_sql("""
                    /*header info*/
                    select c.gvkey, f.cusip, f.datadate, f.fyear, c.cik, substr(c.sic,1,2) as sic2, c.sic, c.naics,
                    
                    /*firm variables*/
                    /*income statement*/
                    f.sale, f.revt, f.cogs, f.xsga, f.dp, f.xrd, f.xad, f.ib, f.ebitda,
                    f.ebit, f.nopi, f.spi, f.pi, f.txp, f.ni, f.txfed, f.txfo, f.txt, f.xint,
                    
                    /*CF statement and others*/
                    f.capx, f.oancf, f.dvt, f.ob, f.gdwlia, f.gdwlip, f.gwo, f.mib, f.oiadp, f.ivao,
                    
                    /*assets*/
                    f.rect, f.act, f.che, f.ppegt, f.invt, f.at, f.aco, f.intan, f.ao, f.ppent, f.gdwl, f.fatb, f.fatl,
                    
                    /*liabilities*/
                    f.lct, f.dlc, f.dltt, f.lt, f.dm, f.dcvt, f.cshrc, 
                    f.dcpstk, f.pstk, f.ap, f.lco, f.lo, f.drc, f.drlt, f.txdi,
                    
                    /*equity and other*/
                    f.ceq, f.scstkc, f.emp, f.csho, f.seq, f.txditc, f.pstkrv, f.pstkl, f.np, f.txdc, f.dpc, f.ajex,
                    
                    /*market*/
                    abs(f.prcc_f) as prcc_f
                    
                    from comp.funda as f
                    left join comp.company as c
                    on f.gvkey = c.gvkey
                    
                    /*get consolidated, standardized, industrial format statements*/
                    where f.indfmt = 'INDL' 
                    and f.datafmt = 'STD'
                    and f.popsrc = 'D'
                    and f.consol = 'C'
                    and f.datadate >= '01/01/2022'
                    and f.datadate <= '12/31/2022'
                    """)


# convert datadate to date fmt
comp['datadate'] = pd.to_datetime(comp['datadate'])

# sort and clean up
comp = comp.sort_values(by=['gvkey', 'datadate']).drop_duplicates()

# clean up csho
comp['csho'] = np.where(comp['csho'] == 0, np.nan, comp['csho'])

# calculate Compustat market equity
comp['mve_f'] = comp['csho'] * comp['prcc_f']

# do some clean up. several variables have lots of missing values
condlist = [comp['drc'].notna() & comp['drlt'].notna(),
            comp['drc'].notna() & comp['drlt'].isnull(),
            comp['drlt'].notna() & comp['drc'].isnull()]
choicelist = [comp['drc']+comp['drlt'],
              comp['drc'],
              comp['drlt']]
comp['dr'] = np.select(condlist, choicelist, default=np.nan)

condlist = [comp['dcvt'].isnull() & comp['dcpstk'].notna() & comp['pstk'].notna() & comp['dcpstk'] > comp['pstk'],
            comp['dcvt'].isnull() & comp['dcpstk'].notna() & comp['pstk'].isnull()]

choicelist = [comp['dcpstk']-comp['pstk'],
              comp['dcpstk']]
comp['dc'] = np.select(condlist, choicelist, default=np.nan)
comp['dc'] = np.where(comp['dc'].isnull(), comp['dcvt'], comp['dc'])

comp['xint0'] = np.where(comp['xint'].isnull(), 0, comp['xint'])
comp['xsga0'] = np.where(comp['xsga'].isnull, 0, 0)

comp['ceq'] = np.where(comp['ceq'] == 0, np.nan, comp['ceq'])
comp['at'] = np.where(comp['at'] == 0, np.nan, comp['at'])

Loading library list...
Done


In [34]:
comp = comp.dropna(subset=['at'])


In [35]:
comp.sort_values(by='datadate')

,gvkey,cusip,datadate,fyear,cik,sic2,sic,naics,sale,revt,...,np,txdc,dpc,ajex,prcc_f,mve_f,dr,dc,xint0,xsga0
394,033425,54738L109,2022-01-31,2021.0,0001701758,25,2510,3371,498.239,498.239,...,0.0,-9.836,7.950,1.0,53.85,814.37355,13.316,0.0,0.179,0
122,009248,778296103,2022-01-31,2021.0,0000745732,56,5651,448140,18916.244,18916.244,...,0.0,15.775,360.664,1.0,97.75,34380.63000,0.000,0.0,89.637,0
437,036896,86646P103,2022-01-31,2021.0,0001643269,73,7370,518210,242.125,242.125,...,0.0,0.000,30.028,1.0,11.91,1355.51283,137.273,0.0,0.174,0
434,036711,63947X101,2022-01-31,2021.0,0001566895,73,7370,518210,273.865,273.865,...,0.0,-24.280,15.785,1.0,45.83,5031.17157,122.687,0.0,1.514,0
125,009599,816850101,2022-01-31,2021.0,0000088941,36,3674,334413,740.858,740.858,...,0.0,-3.782,35.302,1.0,71.10,4557.43890,24.083,0.0,5.091,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,027928,461202103,2022-07-31,2022.0,0000896878,73,7372,513210,12726.000,12726.000,...,0.0,120.000,746.000,1.0,456.17,128608.92044,814.000,0.0,81.000,0
803,171067,697435105,2022-07-31,2022.0,0001327567,73,7370,519130,5501.500,5501.500,...,0.0,NaN,NaN,1.0,499.10,49710.36000,6994.000,NaN,27.400,0
934,190455,40171V100,2022-07-31,2022.0,0001528396,73,7372,511210,812.614,812.614,...,0.0,-54.115,62.387,1.0,77.72,6514.17952,178.276,NaN,19.446,0
44,004036,257651109,2022-07-31,2022.0,0000029644,37,3714,336390,3306.600,3306.600,...,3.7,-1.400,93.800,1.0,54.41,6684.21409,NaN,NaN,14.900,0


In [36]:
### CRSP Block

# Create a CRSP Subsample with Monthly Stock and Event Variables
# Restrictions will be applied later
# Select variables from the CRSP monthly stock and event datasets
crsp = conn.raw_sql("""
                      select a.prc, a.ret, a.retx, a.shrout, a.vol, a.cfacpr, a.cfacshr, a.date, a.permno, a.permco, a.cusip,
                      b.comnam, b.ticker, b.ncusip, b.shrcd, b.exchcd
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date >= '01/01/2022'
                      and a.date <= '12/31/2022'
                      and b.exchcd between 1 and 3
                      """, date_cols=['date'])

In [37]:
crsp

,prc,ret,retx,shrout,vol,cfacpr,cfacshr,date,permno,permco,cusip,comnam,ticker,ncusip,shrcd,exchcd
0,151.690002,-0.039694,-0.039694,19110.0,14878.0,1.0,1.0,2022-01-31,10026.0,7976.0,46603210,J & J SNACK FOODS CORP,JJSF,46603210,11.0,3.0
1,3.970000,-0.024570,-0.024570,26925.0,5821.0,1.0,1.0,2022-01-31,10028.0,7978.0,29402E10,ENVELA CORP,ELA,29402E10,11.0,2.0
2,77.519997,-0.191574,-0.191574,27997.0,28512.0,1.0,1.0,2022-01-31,10032.0,7980.0,72913210,PLEXUS CORP,PLXS,72913210,11.0,3.0
3,7.912000,0.007898,0.007898,6180.0,3664.0,1.0,1.0,2022-01-31,10044.0,7992.0,77467X10,ROCKY MOUNTAIN CHOC FAC INC NEW,RMCF,77467X10,11.0,3.0
4,18.129999,0.000000,0.000000,38709.0,20676.0,1.0,1.0,2022-01-31,10051.0,7999.0,41043F20,HANGER INC,HNGR,41043F20,11.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21306,43.500000,-0.003665,-0.003665,86245.0,333094.0,1.0,1.0,2022-03-31,93423.0,53440.0,83001A10,SIX FLAGS ENTERTAINMENT CORP,SIX,83001A10,11.0,1.0
21307,32.150002,0.023559,0.023559,12612.0,8260.0,1.0,1.0,2022-03-31,93426.0,53443.0,92835K10,VISHAY PRECISION GROUP INC,VPG,92835K10,11.0,1.0
21308,105.129997,0.049935,0.049935,36995.0,33481.0,1.0,1.0,2022-03-31,93427.0,53445.0,G3323L10,FABRINET,FN,G3323L10,12.0,1.0
21309,2.130000,-0.073913,-0.073913,38908.0,173068.0,1.0,1.0,2022-03-31,93434.0,53427.0,78513510,S & W SEED CO,SANW,78513510,11.0,3.0


In [7]:
# change variable format to int
crsp[['permco', 'permno', 'shrcd', 'exchcd']] = crsp[['permco', 'permno', 'shrcd', 'exchcd']].astype(int)

# Line up date to be end of month
crsp['date'] = pd.to_datetime(crsp['date'])
crsp['monthend'] = crsp['date'] + MonthEnd(0)  # set all the date to the standard end date of month

In [9]:
crsp.sort_values(by=['date'])

,prc,ret,retx,shrout,vol,cfacpr,cfacshr,date,permno,permco,cusip,comnam,ticker,ncusip,shrcd,exchcd,monthend
0,151.690002,-0.039694,-0.039694,19110.0,14878.0,1.0,1.0,2022-01-31,10026,7976,46603210,J & J SNACK FOODS CORP,JJSF,46603210,11,3,2022-01-31
4726,271.989990,-0.086393,-0.086393,240163.0,254789.0,1.0,1.0,2022-01-31,64311,21294,65584410,NORFOLK SOUTHERN CORP,NSC,65584410,11,1,2022-01-31
4725,56.070000,-0.196590,-0.196590,257723.0,829501.0,1.0,1.0,2022-01-31,64282,2664,07083010,BATH & BODY WORKS INC,BBWI,07083010,11,1,2022-01-31
4724,468.910004,-0.113659,-0.113659,15395.0,9535.0,1.0,1.0,2022-01-31,64194,20435,16359R10,CHEMED CORP NEW,CHE,16359R10,11,1,2022-01-31
4723,230.460007,0.003615,0.003615,331428.0,353902.0,1.0,1.0,2022-01-31,64186,20357,12552310,CIGNA CORP NEW,CI,12552310,11,1,2022-01-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16555,10.940000,-0.144644,-0.144644,32385.0,3608.0,1.0,1.0,2022-03-31,19172,56990,92262D10,VELOCITY FINANCIAL INC,VEL,92262D10,11,1,2022-03-31
16554,131.570007,0.126653,0.122132,125893.0,302346.0,1.0,1.0,2022-03-31,19166,20681,30249130,F M C CORP,FMC,30249130,11,1,2022-03-31
16553,10.920000,0.228346,0.228346,1000000.0,450055.0,1.0,1.0,2022-03-31,19163,56987,63884N10,NATURA & CO HOLDING S A,NTCO,63884N10,31,1,2022-03-31
16564,10.170000,0.000492,0.000492,30000.0,37305.0,1.0,1.0,2022-03-31,19186,56938,12558Y10,C H P MERGER CORP,CHPM,12558Y10,11,3,2022-03-31


In [3]:




# change variable format to int
crsp[['permco', 'permno', 'shrcd', 'exchcd']] = crsp[['permco', 'permno', 'shrcd', 'exchcd']].astype(int)

# Line up date to be end of month
crsp['date'] = pd.to_datetime(crsp['date'])
crsp['monthend'] = crsp['date'] + MonthEnd(0)  # set all the date to the standard end date of month

crsp = crsp.dropna(subset=['prc'])
crsp['me'] = crsp['prc'].abs() * crsp['shrout']  # calculate market equity

# if Market Equity is Nan then let return equals to 0
crsp['ret'] = np.where(crsp['me'].isnull(), 0, crsp['ret'])
crsp['retx'] = np.where(crsp['me'].isnull(), 0, crsp['retx'])

# impute me
crsp = crsp.sort_values(by=['permno', 'date']).drop_duplicates()
crsp['me'] = np.where(crsp['permno'] == crsp['permno'].shift(1), crsp['me'].fillna(method='ffill'), crsp['me'])

# Aggregate Market Cap
'''
There are cases when the same firm (permco) has two or more securities (permno) at same date.
For the purpose of ME for the firm, we aggregated all ME for a given permco, date.
This aggregated ME will be assigned to the permno with the largest ME.
'''
# sum of me across different permno belonging to same permco a given date
crsp_summe = crsp.groupby(['monthend', 'permco'])['me'].sum().reset_index()
# largest mktcap within a permco/date
crsp_maxme = crsp.groupby(['monthend', 'permco'])['me'].max().reset_index()
# join by monthend/maxme to find the permno
crsp1 = pd.merge(crsp, crsp_maxme, how='inner', on=['monthend', 'permco', 'me'])
# drop me column and replace with the sum me
crsp1 = crsp1.drop(['me'], axis=1)
# join with sum of me to get the correct market cap info
crsp2 = pd.merge(crsp1, crsp_summe, how='inner', on=['monthend', 'permco'])
# sort by permno and date and also drop duplicates
crsp2 = crsp2.sort_values(by=['permno', 'monthend']).drop_duplicates()

### S&P 500 membership
# sp500 = conn.raw_sql("""
#                     select a.*, b.date
#                     from crsp.msp500list as a,
#                     crsp.msf as b
#                     where a.permno=b.permno
#                     and b.date >= a.start and b.date<= a.ending
#                     and b.date>='01/01/1996'
#                     and b.date<='12/31/2021'
#                     order by date;
#                     """, date_cols=['start', 'ending', 'date'])

# Merge with SP500 data
# crsp500 = pd.merge(sp500, crsp2, how = 'left', on = ['permno','date'])

### CCM Block
# * 이 데이터(crsp.ccmxpf_linktable)에 접근 권한이 없음 
# 
# Not Authorized
# Sorry, you do not have access to this Table
# At present, your institution does not have access to this Table. You can view this Table, but you will not be able to access the data.
# 
# You must be subscribed to the following product(s) to access this Table:
# 
# crsp_a_ccm
# Your subscribed products are listed here. If you are interested in access to this data, please contact your local WRDS Representative(s) or contact WRDS Support.

# # merge CRSP and Compustat
# # reference: https://wrds-www.wharton.upenn.edu/pages/support/applications/linking-databases/linking-crsp-and-compustat/
# ccm = conn.raw_sql("""
#                   select gvkey, lpermno as permno, linktype, linkprim, 
#                   linkdt, linkenddt
#                   from crsp.ccmxpf_linktable
#                   where substr(linktype,1,1)='L'
#                   and (linkprim ='C' or linkprim='P')
#                   """)

# ccm['linkdt'] = pd.to_datetime(ccm['linkdt'])
# ccm['linkenddt'] = pd.to_datetime(ccm['linkenddt'])

# # if linkenddt is missing then set to today date
# ccm['linkenddt'] = ccm['linkenddt'].fillna(pd.to_datetime('today'))

# # merge ccm and comp
# ccm1 = pd.merge(comp, ccm, how='left', on=['gvkey'])

# # we can only get the accounting data after the firm public their report
# # for annual data, we use 4, 5 or 6 months lagged data, now we follow Hou, Xue and Zhang (2015) use 4 months lag
# ccm1['yearend'] = ccm1['datadate'] + YearEnd(0)
# ccm1['jdate'] = ccm1['datadate'] + MonthEnd(4)

# # set link date bounds
# ccm2 = ccm1[(ccm1['jdate'] >= ccm1['linkdt']) & (ccm1['jdate'] <= ccm1['linkenddt'])]

# # link comp and crsp
# crsp2 = crsp2.rename(columns={'monthend': 'jdate'})
# data_rawa = pd.merge(crsp2, ccm2, how='inner', on=['permno', 'jdate'])


# #### CCM 데이터 사용 불가능하므로 comp와 crsp2를 cusip 기준으로 merge
# ```python
# # merge ccm and comp
# ccm1 = pd.merge(comp, ccm, how='left', on=['gvkey'])
# 
# # link comp and crsp
# data_rawa = pd.merge(crsp2, ccm2, how='inner', on=['permno', 'jdate'])
# ```


# we can only get the accounting data after the firm public their report
# for annual data, we use 4, 5 or 6 months lagged data, now we follow Hou, Xue and Zhang (2015) use 4 months lag
comp['yearend'] = comp['datadate'] + YearEnd(0)
comp['jdate'] = comp['datadate'] + MonthEnd(4)

# link comp and crsp
comp['cusip'] = comp['cusip'].apply(lambda x: str(x)[:8])
# crsp500 = crsp500.rename(columns={'monthend': 'jdate'})
crsp2 = crsp2.rename(columns={'monthend': 'jdate'})
# data_rawa = pd.merge(crsp500, comp, how='inner', on=['cusip','jdate'])
data_rawa = pd.merge(crsp2, comp, how='inner', on=['cusip','jdate'])
# filter exchcd & shrcd
data_rawa = data_rawa[((data_rawa['exchcd'] == 1) | (data_rawa['exchcd'] == 2) | (data_rawa['exchcd'] == 3)) &
                   ((data_rawa['shrcd'] == 10) | (data_rawa['shrcd'] == 11))]


In [4]:
data_rawa

,prc,ret,retx,shrout,vol,cfacpr,cfacshr,date,permno,permco,...,txdc,dpc,ajex,prcc_f,mve_f,dr,dc,xint0,xsga0,yearend


In [16]:
# process Market Equity
'''
Note: me is CRSP market equity, mve_f is Compustat market equity. Please choose the me below.
'''
data_rawa['me'] = data_rawa['me']/1000  # CRSP ME
# data_rawa['me'] = data_rawa['mve_f']  # Compustat ME

# there are some ME equal to zero since this company do not have price or shares data, 
# we drop these observations
data_rawa['me'] = np.where(data_rawa['me'] == 0, np.nan, data_rawa['me'])
data_rawa = data_rawa.dropna(subset=['me'])


In [13]:
data_rawa

,prc,ret,retx,shrout,vol,cfacpr,cfacshr,date,permno,permco,...,txdc,dpc,ajex,prcc_f,mve_f,dr,dc,xint0,xsga0,yearend


In [12]:
# count single stock years
# data_rawa['count'] = data_rawa.groupby(['gvkey']).cumcount()

# deal with the duplicates
data_rawa.loc[data_rawa.groupby(['datadate', 'permno'], as_index=False).nth([0]).index, 'temp'] = 1


ValueError: cannot set a frame with no defined index and a scalar

In [10]:

# count single stock years
# data_rawa['count'] = data_rawa.groupby(['gvkey']).cumcount()

# deal with the duplicates
data_rawa.loc[data_rawa.groupby(['datadate', 'permno'], as_index=False).nth([0]).index, 'temp'] = 1
data_rawa = data_rawa[data_rawa['temp'].notna()]
data_rawa.loc[data_rawa.groupby(['permno', 'yearend', 'datadate'], as_index=False).nth([-1]).index, 'temp'] = 1
data_rawa = data_rawa[data_rawa['temp'].notna()]

data_rawa = data_rawa.sort_values(by=['permno', 'jdate'])


ValueError: cannot set a frame with no defined index and a scalar